
\ev charging

In [1]:
#Packages
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np
import folium
import math
import random
#from folium.plugins import HeatMap

In [2]:
#Load datasets
dfp = pd.read_csv('Electric_Vehicle_Population_Data.csv') #dataframe for ev population
dfcs = pd.read_csv('Public_EV_Charging_Stations.csv') #dataframe for charging stations
dfa = pd.read_csv('wa.csv', low_memory = False, index_col = 'HASH') #dataframe for addresses

#### Visualizations

In [26]:
m = folium.Map(location=[lat1[0], lon1[0]], zoom_start=13)

for i in range(len(lat1)):
    folium.CircleMarker(
        location=[lat1[i], lon1[i]],
        radius=5,
        color='red',
    ).add_to(m)

for j in range(len(lat2)):
    folium.CircleMarker(
        location=[lat2[j], lon2[j]],
        radius=8,  # Adjust the size of the circle here
        color='blue',
    ).add_to(m)
m

In [ ]:
map_object = folium.Map(location = [47.608013, -122.335167], zoom_start = 11 )
lat1 = [47.6018, 47.57981, 47.581975]
lon1 = [-122.329075, -122.329815, -122.30823]
lat_lon = list(zip(lat1, lon1))
HeatMap(lat_lon).add_to(map_object)
map_object

#### Data Cleaning

In [3]:
#Parameters
postal_codes = ['98104', '98134', '98144']
city = 'Seattle'

In [ ]:
#Ratio
def ratio():
dfcs['Plugs'] = dfcs['EV_Level1_EVSE_Ports'] + dfcs['EV_Level2_EVSE_Ports'] + dfcs['EV_DC_Fast_Ports']
dfcs = dfcs.loc[dfcs['Zipcode'] == '98039']
print(np.sum(dfcs['Plugs']))

dfp.loc[dfp['Postal Code'] == 98039.0].shape[0]

ratio_98104 = 743/124
ratio_98134 = 451/38
ratio_98144 = 1144/28
print(ratio_98104, ratio_98134, ratio_98144)

In [4]:
#dfcs cleaning
dfcs = dfcs.loc[dfcs['Zipcode'].isin(postal_codes)]

dfcs = dfcs.loc[dfcs['Access_Hours'] == '24 hours daily']
dfcs = dfcs.drop(dfcs[dfcs.Restricted_Access == True].index)
dfcs['Plugs'] = dfcs['EV_Level1_EVSE_Ports'] + dfcs['EV_Level2_EVSE_Ports'] + dfcs['EV_DC_Fast_Ports']

In [7]:
dfp = dfp.loc[dfp['City'] == city]
def ev_count(pc):
    n = {}
    for i in pc:
        n[i] = dfp.loc[dfp['Postal Code'] == float(i)].shape[0]
    return n

In [8]:
ev_count(postal_codes)

{'98104': 743, '98134': 451, '98144': 1144}

In [9]:
#Random generating addresses
dfa = dfa.loc[dfa['POSTCODE'].isin(postal_codes)]
dfa = dfa.dropna(subset=['CITY'])
def generate_address(n):
    dfa_inde = list(dfa.index)
    cars_addresses = {f"Car {i}": random.choice(dfa_inde) for i in range(1, n+1)}
    address1 = []
    for car, address in list(cars_addresses.items()):
        address1.append(address)
    return dfa.loc[address1]

In [10]:
cars = generate_address(70)
lat1 = cars['LAT'].tolist()
lon1 = cars['LON'].tolist()
lat2 = dfcs['Latitude'].tolist()
lon2 = dfcs['Longitude'].tolist()

In [11]:
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8 #Radius of the Earth in miles
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [12]:
def distance_matrix(lat1, lon1, lat2, lon2):
    distance = np.zeros((len(lat1), len(lat2)))
    for i in range(len(lat1)):
        for j in range(len(lat2)):
            distance[i, j] = haversine(lon1[i], lat1[i], lon2[j], lat2[j])
    return distance

#### Optimization Model

In [13]:
m = Model("ev_charging")

d = distance_matrix(lat1, lon1, lat2, lon2)
I = range(len(lat1))  # Postal codes
J = range(len(lat2))  # Charging stations
Capacity = dfcs['Plugs'].tolist()

x = {}
for i in I:
    for j in J:
        x[i,j] = m.addVar(vtype=GRB.BINARY, name = f"x_{i}{j}", lb=0)

m.setObjective(quicksum(d[i,j] * x[i, j] for i in I for j in J), GRB.MINIMIZE)

for j in J:
    m.addConstr(quicksum(x[i,j] for i in I) <= Capacity[j])

for i in I:
    m.addConstr(quicksum(x[i,j] for j in J) == 1, f"EV_Assignment_{i}")

m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-17
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 114 rows, 3080 columns and 6160 nonzeros
Model fingerprint: 0xdc42cc17
Variable types: 0 continuous, 3080 integer (3080 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 115.5620608
Presolve time: 0.01s
Presolved: 114 rows, 3080 columns, 6160 nonzeros
Variable types: 0 continuous, 3080 integer (3080 binary)
Found heuristic solution: objective 108.9064906

Root relaxation: objective 9.963536e+01, 788 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Best

#### Solution

In [14]:
if m.status == GRB.OPTIMAL:
    for i in I:
        for j in J:
            if x[i,j].x > 0.5:
                n = haversine(lon1[i], lat1[i], lon2[j], lat2[j])
                p = dfa.loc[(dfa['LON'] == lon1[i]) & (dfa['LAT'] == lat1[i])].iloc[0,8]
                a = dfcs.loc[(dfcs['Longitude'] == lon2[j]) & (dfcs['Latitude'] == lat2[j])].iloc[0,18]
                print(f"Car {i} from postal code {p} traveled {n} miles to charging station at {a}")

Car 0 from postal code 98144 traveled 2.593858842040386 miles to charging station at 583 Battery St
Car 1 from postal code 98144 traveled 1.03708827942188 miles to charging station at 338 Weller St Bridge
Car 2 from postal code 98144 traveled 1.7725099311279395 miles to charging station at 1531 Utah Avenue South
Car 3 from postal code 98144 traveled 2.2940707851713023 miles to charging station at 583 Battery St
Car 4 from postal code 98144 traveled 3.0845475765495736 miles to charging station at 201 S Jackson St
Car 5 from postal code 98134 traveled 0.1525816780362568 miles to charging station at 1st Ave S & S Stacy St
Car 6 from postal code 98104 traveled 0.185466483542743 miles to charging station at 1000 2nd Ave
Car 7 from postal code 98144 traveled 1.506618737499007 miles to charging station at 201 S Jackson St
Car 8 from postal code 98144 traveled 3.2299816873509335 miles to charging station at 201 S Jackson St
Car 9 from postal code 98144 traveled 1.4307056740754667 miles to char